In [ ]:
# 필요 library import
import pandas as pd
import numpy as np
import torch
import joblib

In [ ]:
actual_revenue = pd.read_csv('../../dataset/labels/revenue.csv')
actual_closed = pd.read_csv('../../dataset/labels/closed.csv')

In [ ]:
to_predict_revenue = pd.read_csv('../../dataset/to_predict_data/revenue.csv')
to_predict_closed = pd.read_csv('../../dataset/to_predict_data/closed.csv')

In [ ]:
to_score_revenue = pd.read_csv('../../dataset/to_score_data/revenue.csv')
to_score_closed = pd.read_csv('../../dataset/to_score_data/closed.csv')

In [ ]:
actual_data_closed = actual_closed.drop(columns=['코드'])
actual_code_closed = actual_closed['코드']

In [ ]:
predict_features_revenue_1 = to_predict_revenue.drop(columns=['코드']).values.astype('float32')
score_features_revenue_1 = to_score_revenue.drop(columns=['코드']).values.astype('float32')

In [ ]:
scale_to_100 = [\
                '편의점수',
                '슈퍼마켓수',
                '여관수',
                '외식업수',
                '주유소수',
                '피시방수',
                '당구장수',
                '노래방수',
                '독서실수',
                '지하철역수'
               ]
scale_to_1000 = [\
                 '1인가구',
                 '2인가구',
                 '3인가구',
                 '4인가구',
                
                ]
scale_to_10000 = [\
                  '0-9세주거',
                  '10-19세주거',
                  '20-29세주거',
                  '30-39세주거',
                  '40-49세주거',
                  '50-59세주거',
                  '60-69세주거',
                  '70-79세주거',
                  '80-89세주거',
                  '직장인구',
                  '생활인구'
                 ]
scale_to_100000 = ['임대시세']

for s in scale_to_100:
    to_predict_closed[s] /= 100
    to_predict_revenue[s] /= 100
    to_score_closed[s] /= 100
    to_score_revenue[s] /= 100
    
for s in scale_to_1000:
    to_predict_closed[s] /= 1000
    to_predict_revenue[s] /= 1000
    to_score_closed[s] /= 1000
    to_score_revenue[s] /= 1000

for s in scale_to_10000:
    to_predict_closed[s] /= 10000
    to_predict_revenue[s] /= 10000
    to_score_closed[s] /= 10000
    to_score_revenue[s] /= 10000
    
for s in scale_to_100000:
    to_predict_closed[s] /= 100000
    to_predict_revenue[s] /= 100000
    to_score_closed[s] /= 100000
    to_score_revenue[s] /= 100000

In [ ]:
predict_features_revenue_2 = to_predict_revenue.drop(columns=['코드']).values.astype('float32')
predict_features_closed = to_predict_closed.drop(columns=['코드']).values.astype('float32')
predict_code_revenue = to_predict_revenue['코드']
predict_code_closed = to_predict_closed['코드']
tensor_predict_features_revenue = torch.tensor(predict_features_revenue)
tensor_predict_features_closed = torch.tensor(predict_features_closed)

In [ ]:
score_features_revenue_2 = to_score_revenue.drop(columns=['코드']).values.astype('float32')
score_features_closed = to_score_closed.drop(columns=['코드']).values.astype('float32')
score_code_revenue_2 = to_score_revenue['코드']
score_code_closed = to_score_closed['코드']
tensor_score_features_revenue = torch.tensor(score_features_revenue)
tensor_score_features_closed = torch.tensor(score_features_closed)

In [ ]:
model_revenue_1 = joblib.load('../../models/revenue_1.pkl')
model_revenue_2 = torch.load('../../models/revenue_2.pt')
model_closed = torch.load('../../models/closed.pt')

In [ ]:
predict_revenue_1 = pd.DataFrame(model_revenue_1.predict(predict_features_revenue_1),columns=['expectedprofit2020'])

In [ ]:
tensor_predict_revenue_2 = model_revenue(tensor_predict_features_revenue)
tensor_predict_closed = model_revenue(tensor_predict_features_closed)
tensor_score_revenue_2 = model_revenue(tensor_score_features_revenue)
tensor_score_closed = model_revenue(tensor_score_features_closed)

In [ ]:
predict_revenue_2 = pd.DataFrame(tensor_predict_revenue_2.detach().numpy(),columns=['expectedprofit2020'])
predict_closed = pd.DataFrame(tensor_predict_closed.detach().numpy(),columns=['expectedsurvive2020'])
score_revenue_2 = pd.DataFrame(tensor_score_revenue_2.detach().numpy(),columns=['profitscore'])
score_closed = pd.DataFrame(tensor_score_closed.detach().numpy(),columns=['stablescore'])

In [ ]:
predict_revenue = (predict_revenue_1 + 10000000* predict_revenue_2) /2

In [ ]:
actual_data_survive = 100 - actual_data_closed
actual_survive = pd.concat([actual_code_closed,actual_data_survive], axis=1)
print(actual_survive)

In [ ]:
predict_survive = 100 - predict_closed
score_survive = 100 - score_closed

In [ ]:
predict_survive = pd.concat([predict_code_closed,predict_survive], axis=1)
predict_revenue = pd.concat([predict_code_revenue,predict_revenue], axis=1)

In [ ]:
score_survive = 100 + 10*((score_survive - score_survive.mean()) / score_survive.std())
score_revenue = 100 + 10*((score_revenue - score_revenue.mean()) / score_revenue.std())
survive_score = pd.concat([score_code_closed,score_survive], axis=1)
revenue_score = pd.concat([score_code_revenue,score_revenue], axis=1)

In [ ]:
survive = pd.merge(actual_survive,predict_survive,on='코드',how='outer')
survive = pd.merge(survive,survive_score,on='코드',how='outer')
survive = survive.fillna(0)
print(survive)

In [ ]:
revenue = pd.merge(actual_revenue,predict_revenue,on='코드',how='outer')
revenue = pd.merge(revenue,revenue_score,on='코드',how='outer')
revenue = revenue.fillna(0)
print(revenue)

In [ ]:
data = pd.merge(revenue,survive,on='코드',how='outer')
data = data.rename(columns={'코드':'code','2018월매출':'profit2018','2019월매출':'profit2019','2020월매출':'profit2020'})
data = data.rename(columns={'2018폐업률':'survive2018','2019폐업률':'survive2019','2020폐업률':'survive2020'})
data = data[['code','profitscore','stablescore','profit2018','profit2019','profit2020','expectedprofit2020','survive2018','survive2019','survive2020','expectedsurvive2020']]
data.to_csv("../../result/rank.csv", encoding='utf8')